# Deploying a MedNIST Classifier with MONAI Deploy App SDK

This notebook demos the process of packaging up a trained model using MONAI Deploy App SDK into an artifact which can be run as a local program performing inference, a workflow job doing the same, and a Docker containerized workflow execution.


~~BentoML provides various ways of deploying models with existing platforms like AWS or Azure but we'll focus on local deployment here since researchers are more likely to do this. This tutorial will train a MedNIST classifier like the [MONAI tutorial here](https://github.com/Project-MONAI/tutorials/blob/master/2d_classification/mednist_tutorial.ipynb) and then do the packaging as described in this [BentoML tutorial](https://github.com/bentoml/gallery/blob/master/pytorch/fashion-mnist/pytorch-fashion-mnist.ipynb).~~

## Setup environment

In [1]:
!python -c "import monai" || pip install -q "monai[pillow, tqdm]"
!python -c "import ignite" || pip install -q "monai[ignite]"
!python -c "import gdown" || pip install -q "monai[gdown]"
!python -c "import monai.deploy" || pip install -q "monai-deploy-app-sdk"

## Setup imports

In [2]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import shutil
import tempfile
import glob
import PIL.Image
import torch
import numpy as np

from ignite.engine import Events

from monai.apps import download_and_extract
from monai.config import print_config
from monai.networks.nets import DenseNet121
from monai.engines import SupervisedTrainer
from monai.transforms import (
    AddChannel,
    Compose,
    LoadImage,
    RandFlip,
    RandRotate,
    RandZoom,
    ScaleIntensity,
    EnsureType,
)
from monai.utils import set_determinism

set_determinism(seed=0)

print_config()

MONAI version: 0.6.0
Numpy version: 1.19.5
Pytorch version: 1.9.0
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 0ad9e73639e30f4f1af5a1f4a45da9cb09930179

Optional dependencies:
Pytorch Ignite version: 0.4.5
Nibabel version: 3.2.1
scikit-image version: 0.17.2
Pillow version: 8.3.1
Tensorboard version: 2.6.0
gdown version: 3.13.0
TorchVision version: 0.10.0
ITK version: 5.2.0
tqdm version: 4.62.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.8.0
pandas version: 1.1.5
einops version: 0.3.2

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



## Download dataset

The MedNIST dataset was gathered from several sets from [TCIA](https://wiki.cancerimagingarchive.net/display/Public/Data+Usage+Policies+and+Restrictions),
[the RSNA Bone Age Challenge](http://rsnachallenges.cloudapp.net/competitions/4),
and [the NIH Chest X-ray dataset](https://cloud.google.com/healthcare/docs/resources/public-datasets/nih-chest).

The dataset is kindly made available by [Dr. Bradley J. Erickson M.D., Ph.D.](https://www.mayo.edu/research/labs/radiology-informatics/overview) (Department of Radiology, Mayo Clinic)
under the Creative Commons [CC BY-SA 4.0 license](https://creativecommons.org/licenses/by-sa/4.0/).

If you use the MedNIST dataset, please acknowledge the source.

In [3]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

resource = "https://drive.google.com/uc?id=1QsnnkvZyJPcbRoV_ArW8SnE1OTuoVbKE"
md5 = "0bc7306e7427e00ad1c5526a6677552d"

compressed_file = os.path.join(root_dir, "MedNIST.tar.gz")
data_dir = os.path.join(root_dir, "MedNIST")
if not os.path.exists(data_dir):
    download_and_extract(resource, compressed_file, root_dir, md5)

/tmp/tmp6lgy6bsy


Downloading...
From: https://drive.google.com/uc?id=1QsnnkvZyJPcbRoV_ArW8SnE1OTuoVbKE
To: /tmp/tmprsr1p079/MedNIST.tar.gz
61.8MB [00:06, 10.3MB/s]


Downloaded: /tmp/tmp6lgy6bsy/MedNIST.tar.gz
Verified 'MedNIST.tar.gz', md5: 0bc7306e7427e00ad1c5526a6677552d.
Writing into directory: /tmp/tmp6lgy6bsy.


In [4]:
subdirs = sorted(glob.glob(f"{data_dir}/*/"))

class_names = [os.path.basename(sd[:-1]) for sd in subdirs]
image_files = [glob.glob(f"{sb}/*") for sb in subdirs]

image_files_list = sum(image_files, [])
image_class = sum(([i] * len(f) for i, f in enumerate(image_files)), [])
image_width, image_height = PIL.Image.open(image_files_list[0]).size

print(f"Label names: {class_names}")
print(f"Label counts: {list(map(len, image_files))}")
print(f"Total image count: {len(image_class)}")
print(f"Image dimensions: {image_width} x {image_height}")

Label names: ['AbdomenCT', 'BreastMRI', 'CXR', 'ChestCT', 'Hand', 'HeadCT']
Label counts: [10000, 8954, 10000, 10000, 10000, 10000]
Total image count: 58954
Image dimensions: 64 x 64


## Setup and Train

Here we'll create a transform sequence and train the network, omitting validation and testing since we know this does indeed work and it's not needed here:

In [5]:
train_transforms = Compose(
    [
        LoadImage(image_only=True),
        AddChannel(),
        ScaleIntensity(),
        RandRotate(range_x=np.pi / 12, prob=0.5, keep_size=True),
        RandFlip(spatial_axis=0, prob=0.5),
        RandZoom(min_zoom=0.9, max_zoom=1.1, prob=0.5),
        EnsureType(),
    ]
)

In [6]:
class MedNISTDataset(torch.utils.data.Dataset):
    def __init__(self, image_files, labels, transforms):
        self.image_files = image_files
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        return self.transforms(self.image_files[index]), self.labels[index]


# just one dataset and loader, we won't bother with validation or testing 
train_ds = MedNISTDataset(image_files_list, image_class, train_transforms)
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=300, shuffle=True, num_workers=10)

In [7]:
device = torch.device("cuda:0")
net = DenseNet121(spatial_dims=2, in_channels=1, out_channels=len(class_names)).to(device)
loss_function = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(net.parameters(), 1e-5)
max_epochs = 5

In [8]:
def _prepare_batch(batch, device, non_blocking):
    return tuple(b.to(device) for b in batch)


trainer = SupervisedTrainer(device, max_epochs, train_loader, net, opt, loss_function, prepare_batch=_prepare_batch)


@trainer.on(Events.EPOCH_COMPLETED)
def _print_loss(engine):
    print(f"Epoch {engine.state.epoch}/{engine.state.max_epochs} Loss: {engine.state.output[0]['loss']}")


trainer.run()

Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448272031/work/c10/core/TensorImpl.h:1156.)


Epoch 1/5 Loss: 0.1811893731355667
Epoch 2/5 Loss: 0.08026652783155441
Epoch 3/5 Loss: 0.05008228123188019
Epoch 4/5 Loss: 0.01724996417760849
Epoch 5/5 Loss: 0.029151903465390205


The network will be saved out here as a Torchscript object named `classifier.zip`

In [9]:
torch.jit.script(net).save("classifier.zip")

## Packaging Inference Appplication with MONAI Deploy App SDK

~~BentoML provides it's platform through an API to wrap service requests as method calls. This is obviously similar to how Flask works (which is one of the underlying technologies used here), but on top of this is provided various facilities for storing the network (artifacts), handling the IO component of requests, and caching data. What we need to provide is a script file to represent the services we want, BentoML will take this with the artifacts we provide and store this in a separate location which can be run locally as well as uploaded to a server (sort of like Docker registries).~~



### Creating Application and Operations and connecting them

~~The script below will create our API which includes MONAI code. The transform sequence needs a special read Transform to turn a data stream into an image, but otherwise the code like what was used above for training. The network is stored as an artifact which in practice is the stored weights in the BentoML bundle. This is loaded at runtime automatically, but instead we could load the Torchscript model instead if we wanted to, in particular if we wanted an API that didn't rely on MONAI code. ~~

The script needs to be written out to a file first:

In [11]:
%%writefile mednist_classifier_monaideploy.py

# Copyright 2021 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from monai.deploy.core import (
    Application,
    DataPath,
    ExecutionContext,
    Image,
    InputContext,
    IOType,
    Operator,
    OutputContext,
    env,
    input,
    output,
    resource,
)
from monai.transforms import AddChannel, Compose, EnsureType, ScaleIntensity

MEDNIST_CLASSES = ["AbdomenCT", "BreastMRI", "CXR", "ChestCT", "Hand", "HeadCT"]


@input("image", DataPath, IOType.DISK)
@output("image", Image, IOType.IN_MEMORY)
@env(pip_packages=["pillow"])
class LoadPILOperator(Operator):
    """Load image from the given input (DataPath) and set numpy array to the output (Image)."""

    def compute(self, input: InputContext, output: OutputContext, context: ExecutionContext):
        import numpy as np
        from PIL import Image as PILImage

        input_path = input.get().path

        image = PILImage.open(input_path)
        image = image.convert("L")  # convert to greyscale image
        image_arr = np.asarray(image)

        output_image = Image(image_arr)  # create Image domain object with a numpy array
        output.set(output_image)


@input("image", Image, IOType.IN_MEMORY)
@output("output", DataPath, IOType.DISK)
@env(pip_packages=["monai"])
class MedNISTClassifierOperator(Operator):
    """Classifies the given image and returns the class name."""

    @property
    def transform(self):
        return Compose([AddChannel(), ScaleIntensity(), EnsureType()])

    def compute(self, input: InputContext, output: OutputContext, context: ExecutionContext):
        import json

        import torch

        img = input.get().asnumpy()  # (64, 64), uint8
        image_tensor = self.transform(img)  # (1, 64, 64), torch.float64
        image_tensor = image_tensor[None].float()  # (1, 1, 64, 64), torch.float32

        # Comment below line if you want to do CPU inference
        image_tensor = image_tensor.cuda()

        model = context.models.get()  # get a TorchScriptModel object
        # Uncomment the following line if you want to do CPU inference
        # model.predictor = torch.jit.load(model.path, map_location="cpu").eval()

        with torch.no_grad():
            outputs = model(image_tensor)

        _, output_classes = outputs.max(dim=1)

        result = MEDNIST_CLASSES[output_classes[0]]  # get the class name
        print(result)

        # Get output (folder) path and create the folder if not exists
        output_folder = output.get().path
        output_folder.mkdir(parents=True, exist_ok=True)

        # Write result to "output.json"
        output_path = output_folder / "output.json"
        with open(output_path, "w") as fp:
            json.dump(result, fp)


@resource(cpu=1, memory="1Gi")
class App(Application):
    """Application class for the MedNIST classifier."""

    def compose(self):
        load_pil_op = LoadPILOperator()
        classifier_op = MedNISTClassifierOperator()

        self.add_flow(load_pil_op, classifier_op)


if __name__ == "__main__":
    App(do_run=True)

Writing mednist_classifier_monaideploy.py


### Executing app locally

In [12]:
print(f"Test input file path: {image_files[0][0]}")

Test input file path: /tmp/tmp6lgy6bsy/MedNIST/AbdomenCT/007000.jpeg


In [13]:
!python mednist_classifier_monaideploy.py -i {image_files[0][0]} -o output -m classifier.zip

Going to initiate execution of operator LoadPILOperator
Executing operator LoadPILOperator (Process ID: 24083, Operator ID: 68b38511-63d1-4a73-a438-a675b66262d2)
Done performing execution of operator LoadPILOperator

Going to initiate execution of operator MedNISTClassifierOperator
Executing operator MedNISTClassifierOperator (Process ID: 24083, Operator ID: a82bb8c3-b4c3-43d0-8ec1-3e8c7b79b971)
Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448272031/work/c10/core/TensorImpl.h:1156.)
AbdomenCT
Done performing execution of operator MedNISTClassifierOperator



In [14]:
!cat output/output.json

"AbdomenCT"

### Packaging app

In [15]:
!monai-deploy package mednist_classifier_monaideploy.py --tag mednist_app:latest --model classifier.zip -l DEBUG

[2021-09-11 01:01:42,809] [DEBUG] (app_packager) - FROM nvcr.io/nvidia/pytorch:21.07-py3

    ARG MONAI_GID=1000
    ARG MONAI_UID=1000

    LABEL base="nvcr.io/nvidia/pytorch:21.07-py3"
    LABEL tag="mednist_app:latest"
    LABEL version="0.0.0"
    LABEL sdk_version="0.1.0-a.2"

    ENV DEBIAN_FRONTEND=noninteractive
    ENV TERM=xterm-256color
    ENV MONAI_INPUTPATH=/var/monai/input
    ENV MONAI_OUTPUTPATH=/var/monai/output
    ENV MONAI_WORKDIR=/var/monai
    ENV MONAI_APPLICATION=/opt/monai/app
    ENV MONAI_TIMEOUT=0

    RUN apt update \
     && apt upgrade -y --no-install-recommends \
     && apt install -y --no-install-recommends \
        curl \
        unzip \
     && apt autoremove -y \
     && rm -rf /var/lib/apt/lists/*
    
    USER root

    RUN pip install --no-cache-dir --upgrade setuptools==57.4.0 pip==21.2.4 wheel==0.37.0

    RUN groupadd -g $MONAI_GID -o -r monai
    RUN useradd -g $MONAI_GID -u $MONAI_UID -m -o -r monai

    RUN mkdir -p /etc/monai/ && chown -

\[2021-09-11 01:01:43,364] [DEBUG] (app_packager) -  ---> 8030ed820d03

|[2021-09-11 01:01:43,364] [DEBUG] (app_packager) - Step 23/34 : COPY --chown=monai:monai ./pip/requirements.txt /tmp/requirements.txt

/[2021-09-11 01:01:43,465] [DEBUG] (app_packager) -  ---> 1c6a3a4f960c

-[2021-09-11 01:01:43,465] [DEBUG] (app_packager) - Step 24/34 : RUN curl https://globalcdn.nuget.org/packages/monai.deploy.executor.0.1.0-prealpha.0.nupkg -o /opt/monai/executor/executor.zip      && unzip /opt/monai/executor/executor.zip -d /opt/monai/executor/executor_pkg      && mv /opt/monai/executor/executor_pkg/lib/native/linux-x64/* /opt/monai/executor      && rm -f /opt/monai/executor/executor.zip      && rm -rf /opt/monai/executor/executor_pkg      && chown -R monai:monai /opt/monai/executor      && chmod +x /opt/monai/executor/monai-exec

\[2021-09-11 01:01:43,497] [DEBUG] (app_packager) -  ---> Running in 6d46361fe03f

|[2021-09-11 01:01:44,022] [DEBUG] (app_packager) -   % Total    % Received % Xfer

In [16]:
!docker image ls | grep mednist_app

mednist_app                                                             latest                                   cdfa11aac8ff        2 seconds ago       15.2GB


### Executing packaged app locally

In [17]:
!monai-deploy run mednist_app:latest {image_files[0][0]} output

Checking dependencies...
--> Verifying if "docker" is installed...

--> Verifying if "mednist_app:latest" is available...

Checking for MAP "mednist_app:latest" locally
"mednist_app:latest" found.

Reading MONAI App Package manifest...
 > export '/var/run/monai/export/' detected
Going to initiate execution of operator LoadPILOperator
Executing operator LoadPILOperator (Process ID: 1, Operator ID: fea0dd89-0869-43cc-b4bc-af3bb11778d3)
Done performing execution of operator LoadPILOperator

Going to initiate execution of operator MedNISTClassifierOperator
Executing operator MedNISTClassifierOperator (Process ID: 1, Operator ID: 5ef16659-20bc-4a14-8011-021c1fca852c)
AbdomenCT
Done performing execution of operator MedNISTClassifierOperator



In [18]:
!cat output/output.json

"AbdomenCT"

**Note**: Please execute the following script once the exercise is done.

In [19]:
# Remove data files which is in the temporary folder
if directory is None:
    shutil.rmtree(root_dir)